In [1]:
!pip install -q accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.2 MB/s eta 0:00:00


In [2]:
!pip install -q einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from einops import rearrange, repeat, reduce
import torch.nn.functional as F

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
model_name = 'microsoft/phi-2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
prompt = "Write a story about a Lonely Boy."
tokens = tokenizer.tokenize(prompt)
print(tokens)

['Write', 'Ġa', 'Ġstory', 'Ġabout', 'Ġa', 'ĠLonely', 'ĠBoy', '.']


In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    # Quant Type
    # We will use the "nf4" format which was introduced in the QLoRA paper
    bnb_4bit_quant_type = "nf4",
    # As the model weights are stored in 4 bits and when we want to compute its only going to use 16 bits so we have more accuracy
    bnb_4bit_compute_dype = torch.float16,
    # Quantization parameters are quantized
    bnb_4bit_use_double_quant = False,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    # use_flash_attention_2 = True,  # phi doesnt support yet
    trust_remote_code = True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23",
)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [9]:
model.eval()

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [10]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
print(input_ids)

tensor([[16594,   257,  1621,   546,   257, 46501,  6387,    13]],
       device='cuda:0')


## Greedy Decoding
At each time step, when we input our sequence into the model, we receive a probability distribution across the entire vocabulary. Here we consistently select the token with the highest probability at every step.

In [11]:
def greedy_decoding(input_ids, max_tokens=300):
  with torch.inference_mode():
    for _ in range(max_tokens):
      # input_ids shape: [1, current_seq_len]
      outputs = model(input_ids)

      # outputs.logits shape: [1, current_seq_len, vocab_size]
      next_token_logits = outputs.logits[:,-1,:]
      # next_token_logits shape: [1, vocab_size]

      next_token = torch.argmax(next_token_logits, dim=-1)
      # next_token shape: [1] (Most Probable next token ID)

      # Stop generation if the model produces </s> token
      if next_token == tokenizer.eos_token_id:
        break

      # rearrange(next_token, "c -> 1 c"): changes shape to [1, 1] for concatenation
      input_ids = torch.cat([input_ids, rearrange(next_token, 'c -> 1 c')], dim=-1)
      # input_ids shape after concat: [1, current_seq_len + 1]

    generated_text = tokenizer.decode(input_ids[0])
    # input_ids[0] shape for decoding: [current_sequence_length]

  return generated_text

In [12]:
print(greedy_decoding(input_ids))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Write a story about a Lonely Boy.

Answer: Once upon a time, there was a boy named Jack who lived in a big city. He had no friends and spent most of his time alone. One day, he decided to take a walk in the park to clear his mind. As he was walking, he saw a group of kids playing and having fun. He wanted to join them, but he was too shy and didn't know how to approach them. He sat on a bench and watched them play, feeling lonely and sad. Suddenly, a girl named Lily came and sat next to him. She noticed that he looked sad and asked him what was wrong. Jack opened up to her and told her about his loneliness. Lily listened and then invited him to join their game. Jack was hesitant at first, but he decided to give it a try. To his surprise, he had a great time playing with the other kids and even made some new friends. From that day on, Jack no longer felt lonely and was grateful to Lily for showing him that there are always people who care.

Exercise: Have a conversation with a friend ab

## Beam Search
At each time step, it explores multiple possible sequences of words simultaneously, keeping only a fixed number of top candidates at each step (known as the beam size). This approach helps to alleviate the issue of generating overly repetitive or implausible sequences by considering a diverse set of alternatives, ultimately selecting the most likely sequence based on a scoring function such as the model's probability distribution or a combination of various factors.

In [13]:
def beam_search(input_ids, max_tokens=100, beam_size=2):
  # Initialize the scores for each beam with zeros. Shape: [beam_size]
  beam_scores = torch.zeros(beam_size).to(device)

  # Duplicate the initial sequence for each beam. Shape: [beam_size, seq_len]
  beam_sequences = input_ids.clone()

  # Create a boolean mask to keep track of active beams. Shape: [beam_size]
  active_beams = torch.ones(beam_size, dtype=torch.bool)

  for step in range(max_tokens):
    # The model is expected to handle batched input, hence the shape of beam_sequences is [beam_size, current_seq_len]
    outputs = model(beam_sequences)

    # Extract the last logits from the output to get probs for the next token. Shape: [beam_size, vocab_size]
    logits = outputs.logits[:, -1, :]
    probs = F.softmax(logits, dim=-1)

    # Calculate the score for each beam and token by flattening the probabilities and selecting the top ones.
    # The flattened shape is [beam_size * vocab_size], from which we select the top beam_size scores.
    top_scores, top_indices = torch.topk(probs.flatten(), k=beam_size, sorted=False)

    # Map flat indices back to beam and token indices
    # beam_indices is the index in the beam, shape: [beam_size]
    # token_indices is the index of the token in the vocab, shape: [beam_size]
    beam_indices = top_indices // probs.shape[-1]
    token_indices = top_indices % probs.shape[-1]

    # Update the sequences with the new tokens at the end, shape after update: [beam_size, current_seq_len + 1]
    # This concats the best token for each beam to the end of the sequences
    beam_sequences = torch.cat([
        beam_sequences[beam_indices],
        token_indices.unsqueeze(-1)
    ], dim=-1)

    # Update the beam scores with the top scores. Shape: [beam_size]
    beam_scores = top_scores

    # Check for the end of seq tokens and update the active beams
    # If a beam produces an EOS token, it is marked as inactive
    active_beams = ~(token_indices == tokenizer.eos_token_id)

    # If all beams are inactive, exit the loop
    if not active_beams.any():
      print("No Active Beams")
      break

  # Select the beam with the highest score as the best seq, shape: [best_seq_len]
  best_beam = beam_scores.argmax()
  best_sequence = beam_sequences[best_beam]

  # Decode the best sequence to generate the final text
  generated_text = tokenizer.decode(best_sequence)

  return generated_text

In [14]:
print(beam_search(input_ids))

Write a story about a Lonely Boy.

Answer: Once upon a time, there was a boy named Jack. He lived in a big city with his parents, but he always felt lonely. He didn't have many friends at school and his parents were always busy with work. Jack spent most of his time alone, playing video games or reading books.

One day, Jack's parents decided to take him on a trip to the countryside. Jack was excited to see new places and maybe make some new friends. But when they


## Top-K Sampling
The concept of Top-K sampling involves sampling only from the top K tokens in the probability distribution. Let's say we set K to 50; this means that when we predict the next token, we sort the entire vocabulary according to the softmax scores of our logits. Then, we retain only the top 50 options from the vocabulary and remove the rest of the items. Subsequently, we randomly select a token from the previously filtered list.

In [15]:
def top_k_sampling(input_ids, max_tokens=200, top_k=50):
  with torch.inference_mode():
    for _ in range(max_tokens):
      outputs = model(input_ids)

      # Extract the logits of next token
      next_token_logits = outputs.logits[:, -1, :]

      # Select the top K tokens from the prob distribution
      top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)

      # Apply softmax to convert logits to probs
      top_k_probs = F.softmax(top_k_logits, dim=-1)

      # Sample from the top K tokens to determine the next token
      next_token_index = torch.multinomial(top_k_probs, num_samples=1)

      # Map the samples token back to its original index in the logits tensor
      next_token = top_k_indices.gather(-1, next_token_index)

      # Concat the new token to the seq of input ids
      input_ids = torch.cat([input_ids, next_token], dim=-1)

    generated_text = tokenizer.decode(input_ids[0])

  return generated_text

In [16]:
print(top_k_sampling(input_ids))

Write a story about a Lonely Boy. (15-20 words each)

Topic: Daily

Use Case 1: A lonely student named Jack moves to a new school and feels out of place. He meets a new friend, Sam, who tells him about the Lonely Boy books and how they helped him cope with his own loneliness. Jack decides to check out the books from the library and feels more understanding and connected to the characters. He also decides to start a book club at school to share his love for these books with others.

Use Case 2: A group of friends are discussing their feelings of loneliness and how it affects their daily lives. One friend recommends the Lonely Boy books to the group and they decide to read all of them together. As they discuss the books, they are able to open up to each other and form a deeper bond. They also realize that they are not alone in their struggles and can find comfort in each other's company.

Topic: Education

Use Case 1: A teacher


## Top-P Nucleus Sampling
Instead of selecting a fixed number of top tokens (like the top 50 in Top-K sampling), Top-P sampling chooses tokens based on their cumulative probability. It looks at the list of all possible next tokens, ranked by their probability, and adds them up from the most likely to the least likely until the total probability reaches a threshold P (like 0.9 or 90%).

In [17]:
def top_p_sampling(input_ids, max_tokens=100, top_p=0.95):
    # Temporarily disables gradient calculation to improve performance and reduce memory usage
    with torch.inference_mode():
        for _ in range(max_tokens):
                # Obtain the model's outputs based on the input IDs
                outputs = model(input_ids)
                next_token_logits = outputs.logits[:, -1, :]

                # Sort the logits in descending order and apply softmax to get probabilities
                sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
                sorted_probabilities = F.softmax(sorted_logits, dim=-1) # Apply temperature

                # Calculate cumulative probabilities
                cumulative_probs = torch.cumsum(sorted_probabilities, dim=-1)

                # Identify and remove tokens with cumulative probability above the threshold (top_p)
                # Ensuring the first token is always selected by setting the first position to False
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 0] = False  # Keep the most probable token

                # Get the actual indices to remove from the original logits tensor
                indices_to_remove = sorted_indices[sorted_indices_to_remove]

                # Set the logits of removed tokens to -infinity to exclude them from sampling
                next_token_logits.scatter_(-1, indices_to_remove[None, :], float('-inf'))

                # Re-calculate probabilities after filtering and sample from this distribution
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)

                # Append the sampled token to the input IDs for the next iteration
                input_ids = torch.cat([input_ids, next_token], dim=-1)

        # Decode the tensor of input IDs to a string of text
        generated_text = tokenizer.decode(input_ids[0])

    return generated_text

In [18]:
print(top_p_sampling(input_ids, max_tokens=300))

Write a story about a Lonely Boy.
 
Some possible answers:
- Chris was always lonely. He didn't have any friends at school and his family was always busy. He spent most of his time alone in his room, reading books or playing video games. One day, he decided to explore the woods behind his house and that's when he met a lonely monkey who lived alone in a tree. They became fast friends and Chris finally felt like he had a companion.
- Emily's family was moving to a new city and she was very sad. On the day of the move, she was sitting alone in her room when she saw a lonely bird who was building a nest outside her window. She decided to make a new home for the bird and they became best friends. Now Emily always has a friend to keep her company when she feels lonely.

2. Rewrite the song lyrics to include your own personal experiences with loneliness.

Example:
"Without You" (with added lyrics)

Oh, I miss you so much 
I wish you were here with me 
Being alone is just so tough 
Every day,